# PPatel Homework #4 - Pandas(PyCitySchools) 

In [1]:
#DEPENDANCIES & SETUP-------------------------------------------------------------------------------------
import pandas as pd
import numpy as np

school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#school_data_complete.shape

In [2]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [3]:
total_schools = len(school_data_complete["school_name"].unique())
total_students = len(school_data_complete["student_name"])
school_budgets = sum(school_data_complete["budget"].unique())

math_data = school_data_complete["math_score"]
reading_data = school_data_complete["reading_score"]
passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]

ave_math = math_data.mean()
ave_reading = reading_data.mean()
overall_passing_rate = (ave_math+ave_reading)/2
percent_passing_math = (len(passing_math)/total_students)*100
percent_passing_reading = (len(passing_reading)/total_students)*100

#BUILD DATAFRAME----------------------------------------------------------------------------------------
district_summary = pd.DataFrame({"Total Schools":[total_schools],
                               "Total Students":[total_students],
                               "Total Budget":[school_budgets],
                               "Average Math Score":[ave_math],
                               "Average Reading Score":[ave_reading],
                               "% Passing Math":[percent_passing_math],
                               "% Passing Reading":[percent_passing_reading],
                               "Overall Passing Rate":[overall_passing_rate]})

# LONG FORM FORMATING, DATAFRAME COLUMNS----------------------------------------------------------------
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map("{:.2f}%".format)

#PRINT FINAL DATAFRAME-----------------------------------------------------------------------------------
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,80.43%


# Summary by School

In [7]:
#SET NEW INDEX-----------------------------------------------------------------------------------------
school_data_complete.columns = ['Student ID', 'student_name', 'gender', 'grade', 'School Name', 'reading_score', 'math_score', 'School ID', 'School Type', 'size', 'budget']
data_by_schools = school_data_complete.set_index(['School Name'])

#CALCULATE VARIABLES-----------------------------------------------------------------------------------
schools_grouped = data_by_schools.groupby(["School Name"])
school_types = schools_grouped['School Type'].unique()
students_per_school = schools_grouped["Student ID"].count()
school_budgets = schools_grouped["budget"].mean()
budget_per_student = school_budgets/students_per_school
ave_math_by_school = schools_grouped['math_score'].mean()
ave_reading_by_school = schools_grouped['reading_score'].mean()

passing_math_grouped = passing_math.groupby(["school_name"])
total_passing_math_by_school = passing_math_grouped["math_score"].count()
percent_passing_math_by_school = total_passing_math_by_school/students_per_school

passing_reading_grouped = passing_reading.groupby(["school_name"])
total_passing_reading_by_school = passing_reading_grouped["reading_score"].count()
percent_passing_reading_by_school = total_passing_reading_by_school/students_per_school

OPR_by_school = (percent_passing_math_by_school+percent_passing_reading_by_school)/2

#BUILD DATAFRAME & FORMAT DATAFRAME COLUMNS-------------------------------------------------------------
###CANNOT figure out how to format Types variable in the DF to not show up as a list
summary_by_school = pd.DataFrame({"School Type": school_types,
                            "Total Students": students_per_school,
                            "Total School Budget": school_budgets.map("${:,}".format),
                            "Budget per Student": budget_per_student.map("${:,.2f}".format),
                            "Average Math Score": ave_math_by_school.map("{:.2f}".format),
                            "Average Reading Score": ave_reading_by_school.map("{:.2f}".format),
                            "% Passing Math":(percent_passing_math_by_school*100).map("{:.2f}%".format),
                            "% Passing Reading":(percent_passing_reading_by_school*100).map("{:.2f}%".format),
                            "% Overall Pass Rate":(OPR_by_school*100).map("{:.2f}%".format)})

summary_by_school

,School Type,Total Students,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Bailey High School,[District],4976,"$3,124,928",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,[Charter],1858,"$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,[District],2949,"$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,[District],2739,"$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,[Charter],1468,"$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,[District],4635,"$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,[Charter],427,"$248,087",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,[District],2917,"$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,[District],4761,"$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Pena High School,[Charter],962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%


# Top Performing Schools (by Passing Rate)

In [8]:
school_OPR_descending = summary_by_school.sort_values("% Overall Pass Rate", ascending=False)
top_five_OPR = school_OPR_descending.head(5)
top_five_OPR

,School Type,Total Students,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Cabrera High School,[Charter],1858,"$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,[Charter],1635,"$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,[Charter],1468,"$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,[Charter],962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,[Charter],2283,"$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.20%


# Bottoming Performing Schools (by Passing Rate)

In [9]:
school_OPR_ascending = summary_by_school.sort_values("% Overall Pass Rate")
bottom_five_OPR = school_OPR_ascending.head(5)
bottom_five_OPR

,School Type,Total Students,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Rodriguez High School,[District],3999,"$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,[District],2949,"$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,[District],2917,"$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,[District],4761,"$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,[District],2739,"$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.80%


# School Breakdown: Average Math Scores by Grade

In [11]:
#CALCULATE VARIABLES-----------------------------------------------------------------------------------
ninth_grade_data = school_data_complete.loc[school_data_complete["grade"] == "9th"]
ninthgrouped = ninth_grade_data.groupby(["School Name"])
ninthmath_by_school = ninthgrouped["math_score"].mean()
#print(ninthmath_by_school)

tenth_grade_data = school_data_complete.loc[school_data_complete["grade"] == "10th"]
tenthgrouped = tenth_grade_data.groupby(["School Name"])
tenthmath_by_school = tenthgrouped["math_score"].mean()

eleventh_grade_data = school_data_complete.loc[school_data_complete["grade"] == "11th"]
eleventhgrouped = eleventh_grade_data.groupby(["School Name"])
eleventhmath_by_school = eleventhgrouped["math_score"].mean()

twelfth_grade_data = school_data_complete.loc[school_data_complete["grade"] == "12th"]
twelfthgrouped = twelfth_grade_data.groupby(["School Name"])
twelfthmath_by_school = twelfthgrouped["math_score"].mean()

#BUILD DATAFRAME & FORMAT DATAFRAME COLUMNS------------------------------------------------------------
SchoolMath_by_grade = pd.DataFrame({"9th Grade": ninthmath_by_school.map("{:.2f}".format),
                        "10th Grade": tenthmath_by_school.map("{:.2f}".format),
                        "11th Grade": eleventhmath_by_school.map("{:.2f}".format),
                        "12th Grade": twelfthmath_by_school.map("{:.2f}".format)})

SchoolMath_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


# School Breakdown: Average Reading Scores by Grade

In [13]:
#CALCULATE VARIABLES-----------------------------------------------------------------------------------
ninthreading_by_school = ninthgrouped["reading_score"].mean()
tenthreading_by_school = tenthgrouped["reading_score"].mean()
eleventhreading_by_school = eleventhgrouped["reading_score"].mean()
twelfthreading_by_school = twelfthgrouped["reading_score"].mean()

#BUILD DATAFRAME---------------------------------------------------------------------------------------
SchoolReading_by_grade = pd.DataFrame({"9th Grade": ninthreading_by_school.map("{:.2f}".format),
                        "10th Grade": tenthreading_by_school.map("{:.2f}".format),
                        "11th Grade": eleventhreading_by_school.map("{:.2f}".format),
                        "12th Grade": twelfthreading_by_school.map("{:.2f}".format)})

SchoolReading_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


# Scores by School Spending

In [14]:
budget_per_student

School Name
Bailey High School       628.0
Cabrera High School      582.0
Figueroa High School     639.0
Ford High School         644.0
Griffin High School      625.0
Hernandez High School    652.0
Holden High School       581.0
Huang High School        655.0
Johnson High School      650.0
Pena High School         609.0
Rodriguez High School    637.0
Shelton High School      600.0
Thomas High School       638.0
Wilson High School       578.0
Wright High School       583.0
dtype: float64

In [19]:
school_summary_short = pd.DataFrame({"Average Budget per Student": budget_per_student,
                            "Average Math Score": ave_math_by_school.map("{:.2f}".format),
                            "Average Reading Score": ave_reading_by_school.map("{:.2f}".format),
                            "% Passing Math":(percent_passing_math_by_school*100).map("{:.2f}%".format),
                            "% Passing Reading":(percent_passing_reading_by_school*100).map("{:.2f}%".format),
                            "% Overall Pass Rate":(OPR_by_school*100).map("{:.2f}%".format)})

# CREATE BINS------------------------------------------------------------------------------------------
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary_short["Spending Ranges"] = pd.cut(school_summary_short["Average Budget per Student"], spending_bins, labels=group_names)
scores_by_spending = school_summary_short.groupby("Spending Ranges")

school_summary_short["Average Budget per Student"] = school_summary_short["Average Budget per Student"].map("${:,.2f}".format)
scores_by_spending.max()

,Average Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Spending Ranges,,,,,,
<$585,$583.00,83.80,83.99,94.13%,97.04%,95.59%
$585-615,$609.00,83.84,84.04,94.59%,95.95%,95.27%
$615-645,$644.00,83.42,83.85,93.39%,97.31%,95.29%
$645-675,$655.00,77.29,81.18,66.75%,81.32%,73.81%


# Scores by School Size

In [20]:
#BUILD DATAFRAME---------------------------------------------------------------------------------------
school_summary_short2 = pd.DataFrame({"Total Students": students_per_school,
                            "Average Math Score": ave_math_by_school.map("{:.2f}".format),
                            "Average Reading Score": ave_reading_by_school.map("{:.2f}".format),
                            "% Passing Math":(percent_passing_math_by_school*100).map("{:.2f}%".format),
                            "% Passing Reading":(percent_passing_reading_by_school*100).map("{:.2f}%".format),
                            "% Overall Pass Rate":(OPR_by_school*100).map("{:.2f}%".format)})

# CREATE BINS------------------------------------------------------------------------------------------
size_bins = [0, 1000, 2000, 5000]
group_names2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary_short2["Size Ranges"] = pd.cut(school_summary_short2["Total Students"], size_bins, labels=group_names2)
scores_by_size = school_summary_short2.groupby("Size Ranges")

scores_by_size.max()

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Size Ranges,,,,,,
Small (<1000),962,83.84,84.04,94.59%,96.25%,95.27%
Medium (1000-2000),1858,83.68,83.98,94.13%,97.31%,95.59%
Large (2000-5000),4976,83.27,83.99,93.87%,96.54%,95.20%


# Scores by School Type

In [21]:
#CALCULATE VARIABLES FOR GROUPING BY SCHOOL TYPE---------------------------------------------------------
types = school_data_complete.groupby(["School Type"])

students_per_type = types["Student ID"].count()
ave_math_by_type = types["math_score"].mean()
ave_reading_by_type = types["reading_score"].mean()

mathpass_index = passing_math.set_index(['type'])
mathpass_by_type = mathpass_index.groupby(['type'])
total_mathpass_by_type = mathpass_by_type["math_score"].count()
percent_mathpass_by_type = total_mathpass_by_type/students_per_type

readpass_index = passing_reading.set_index(['type'])
readpass_by_type = readpass_index.groupby(['type'])
total_readpass_by_type = readpass_by_type["math_score"].count()
percent_readpass_by_type = total_readpass_by_type/students_per_type

OPR_by_type = (percent_mathpass_by_type+percent_readpass_by_type)/2

#BUILD DATAFRAME----------------------------------------------------------------------------------------
school_summary_short3 = pd.DataFrame({"Total Students": students_per_type,
                                    "Average Math Score": ave_math_by_type.map("{:.2f}".format),
                                    "Average Reading Score": ave_reading_by_type.map("{:.2f}".format),
                                    "% Passing Math": (percent_mathpass_by_type*100).map("{:.2f}%".format),
                                    "% Passing Reading": (percent_readpass_by_type*100).map("{:.2f}%".format),
                                    "% Overall Pass Rate": (OPR_by_type*100).map("{:.2f}%".format)})

school_summary_short3

,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Pass Rate
Charter,12194,83.41,83.90,93.70%,96.65%,95.17%
District,26976,76.99,80.96,66.52%,80.91%,73.71%
